# Linear Assignment Problem

In [2]:
# Force keras to run on CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Notebook at full width in the browser
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
from urllib.request import urlretrieve
from pathlib import Path
from collections import defaultdict
from abc import ABC, abstractmethod

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams["image.interpolation"] = "none"
matplotlib.rcParams['figure.figsize'] = (14, 10)
import numpy as np
from tifffile import imread, imwrite
from tqdm.auto import tqdm
import skimage
import pandas as pd
import scipy
import pathlib
import networkx
import pathlib
from tifffile import imread, imwrite
import tensorboard

# from stardist import fill_label_holes, random_label_cmap
# from stardist.plot import render_label
# from stardist.models import StarDist2D
# from stardist import _draw_polygons
# from csbdeep.utils import normalize

# import napari

# lbl_cmap = random_label_cmap()
# # Pretty tqdm progress bars 
# ! jupyter nbextension enable --py widgetsnbextension

/home/llanosp/miniconda3/envs/celltracking/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Calculate Euclidean Distances

## Calculate Centroids

In [3]:
# function to convert 'TRA' channel into cell and frame-wise centroid positions
## Function to extract trajectories from data

base_path = pathlib.Path("/mnt/shared/celltracking/data/cho/")

# read parent-child links from file
links = np.loadtxt(base_path / "01_GT/TRA" / "man_track.txt", dtype=int)

# read annotated image stack
centroids = np.stack([imread(xi) for xi in sorted((base_path / "01_GT/TRA").glob("*.tif"))])  # images

# extract centroids from annotated image stacks
centers = skimage.measure.regionprops(centroids[0,0,:,:])

In [45]:
# function to convert 'TRA' channel into cell and frame-wise centroid positions
## Function to extract trajectories from data

base_path = pathlib.Path("/mnt/shared/celltracking/data/cho/")


# read parent-child links from file
links = np.loadtxt(base_path / "01_GT/TRA" / "man_track.txt", dtype=int)

# read annotated image stack
centroids = np.stack([imread(xi) for xi in sorted((base_path / "01_GT/TRA").glob("*.tif"))])  # images

# extract centroids from annotated image stacks
centers = skimage.measure.regionprops(centroids[0,0,:,:])
tracks = []
for t, frame in enumerate(centroids):
    centers = skimage.measure.regionprops(frame)
    for c in centers:
        tracks.append([c.label, t, int(c.centroid[1]), int(c.centroid[2])])
        
# constructs graph 
tracks = np.array(tracks)

graph = networkx.DiGraph()
for cell_id, t, x, y in tracks:
    graph.add_node((cell_id,t), x=x, y=y, t=t)
    
for cell_id, t in graph.nodes():
    if (cell_id, t+1) in graph.nodes():
        graph.add_edge((cell_id, t), (cell_id,t+1))

for child_id, child_from, _, child_parent_id in links:
    for parent_id, _, parent_to, _ in links:
        if child_parent_id == parent_id:
            graph.add_edge((parent_id, parent_to), (child_id, child_from))
            


In [62]:
def pairwise_euclidian_distance(points0, points1):
    print("Iterative pairwise euclidian distance")
    dists = []
    for p0 in points0:
        for p1 in points1:
            dists.append(np.sqrt(((p0 - p1)**2).sum()))
            
    dists = np.array(dists).reshape(len(points0), len(points1))
    return dists

In [138]:
tracks

array([[  1,   0,  55,  55],
       [  4,   0, 123, 212],
       [  7,   0, 177,  87],
       ...,
       [ 26,  91, 351, 367],
       [ 27,  91, 176,  67],
       [ 28,  91, 389, 432]])

In [51]:
type(tracks)

numpy.ndarray

In [59]:
tracks[0][0]

1

In [72]:
np.where(tracks[:,0]==1)

(array([  0,  10,  20,  30,  39,  48,  57,  66,  75,  84,  93, 102, 111,
        119, 127, 135, 143, 151, 159, 167, 175, 183, 191, 199, 207, 215,
        223, 231, 239, 247, 255, 262, 268, 274, 281]),)

In [143]:
idx=np.where(tracks[:,1]==11)

In [144]:
idx

(array([102, 103, 104, 105, 106, 107, 108, 109, 110]),)

In [100]:
tracks[idxt_next,2]

array([[ 59, 122, 178, 290,  80, 242, 397, 393, 395, 359]])

## Creating matrix t x (t+1)

In [154]:
from scipy.spatial import distance
distances = []
#row corresponds to index of cells in t
rows = [] 
#row corresponds to index of cells in t+1
cols = []

#Loop throug the times frames
for t in range(max(tracks[:,1])):

    #get index
    idxt=np.where(tracks[:,1]==t)[0]
    idxt_next=np.where(tracks[:,1]==t+1)[0]
    t_matrix=np.zeros((len(idxt),len(idxt_next)), dtype=float)

    for ii in range(0, len(idxt)):
        for jj in range(0, len(idxt_next)):
            #coordinate x,y cellN in t
            pt1=[tracks[ii,2], tracks[ii,3]]
            #pt1=tracks[ii,2:]-->embedding
            #coordinate x,y cellN in t next
            pt2=[tracks[jj,2], tracks[jj,3]] 
            #distance from pt1 and pt2
            dist=distance.euclidean(pt1,pt2)
            #dist = np.sqrt(np.sum(np.square(pt1-pt2)))
            #fill matrix with the distances
            
            t_matrix[ii,jj]= dist
            
    #print(t, len(idxt), len(idxt_next))
    distances.append(t_matrix)
    rows.append(idxt)
    cols.append(idxt_next)
    

In [157]:
## Creating matrix t x (t+1)

In [156]:
# extract trajectories from graph set
#tracks = [graph.subgraph(c) for c in networkx.weakly_connected_components(graph) if len(c)>0]

# remove tracks with 0 edges
#tracks = [track for track in tracks if len(track.edges)>0]


#graph = networkx.DiGraph()
#for cell_id, t, x, y in tracks:
#    graph.add_node((cell_id,t), x=x, y=y, t=t)
    
#for cell_id, t in graph.nodes():
#    if (cell_id, t+1) in graph.nodes():
#        graph.add_edge((cell_id, t), (cell_id,t+1))

#for child_id, child_from, _, child_parent_id in links:
#    for parent_id, _, parent_to, _ in links:
#        if child_parent_id == parent_id:
#            graph.add_edge((parent_id, parent_to), (child_id, child_from))
            
# extract trajectories from graph set
#tracks = [graph.subgraph(c) for c in networkx.weakly_connected_components(graph) if len(c)>0]

# remove tracks with 0 edges
#tracks = [track for track in tracks if len(track.edges)>0]
#tracks.label

ValueError: too many values to unpack (expected 4)

In [14]:
import pandas as pd

In [15]:
x = np.stack([imread(xi) for xi in sorted((base_path / "01").glob("*.tif"))])
y = np.stack([imread(xi) for xi in sorted((base_path / "01_GT/TRA").glob("*.tif"))])

assert x.shape == y.shape
print(f"Number of images: {len(x)}")
print(f"Shape of images: {x[0].shape}")
print(f"Shape of images: {y[0].shape}")
#x, y = preprocess(x, y)

Number of images: 92
Shape of images: (5, 443, 512)
Shape of images: (5, 443, 512)


In [18]:
links = np.loadtxt(base_path / "01_GT/TRA" / "man_track.txt", dtype=int)
links = pd.DataFrame(data=links, columns=["track_id", "from", "to", "parent_id"])
print("Links")
links[:20]

Links


,track_id,from,to,parent_id
0,1,0,34,0
1,2,38,39,1
2,3,39,40,1
3,4,0,29,0
4,5,33,91,4
5,7,0,30,0
6,8,34,91,7
7,9,43,91,7
8,10,0,91,0
9,11,0,91,0


In [158]:
## Check birth and death

In [ ]:
ids_from, ids_to = nearest_neighbor(cost_matrix)
    births = np.array(list(set(range(cost_matrix.shape[1])) - set(ids_to)))
    deaths = np.array(list(set(range(cost_matrix.shape[0])) - set(ids_from)))
        
        # Account for +1 offset of the dense labels
    ids_from += 1
    ids_to += 1
    births += 1
    deaths += 1
        
    links = {"links": (ids_from, ids_to), "births": births, "deaths": deaths}
    return links

In [ ]:
 ids_from, ids_to = nearest_neighbor(cost_matrix, self.threshold)
        births = np.array(list(set(range(cost_matrix.shape[1])) - set(ids_to)))
        deaths = np.array(list(set(range(cost_matrix.shape[0])) - set(ids_from)))
        
        # Account for +1 offset of the dense labels
        ids_from += 1
        ids_to += 1
        births += 1
        deaths += 1
        
        links = {"links": (ids_from, ids_to), "births": births, "deaths": deaths}
        return links

Optimal frame-by-frame matching (Linear assignment problem or Weighted bipartite matching)
The nearest neighbor algorithm above will not pick the best solution in many cases. For example, it does not consider the local arrangement of a few detections to create links, something which the human visual system is very good at.

We need a better optimization algorithm to minimize the total minimal linking distance between two frames. To use a classic and efficient optimization algorithm, we will represent this linking problem as a bipartite graph. Here is an example:

In [8]:
class FrameByFrameLinker(ABC):
    """Abstract base class for linking detections by considering pairs of adjacent frames."""
    
    def link(self, detections, images=None):
        """Links detections in t frames.
        
        Args:
        
            detections:
            
                List of t numpy arrays of shape (x,y) with contiguous label ids. Background = 0.
                
            images (optional):
            
                List of t numpy arrays of shape (x,y).
        
        Returns:
        
            Linking dictionary:
                "links":
                    
                    Tuple of lists. Links from frame t to frame t+1 of form (from0, to0) are split up into two lists: 
                    - idgs_from: [from0, from1 , ...])
                    - ids_to: [to0, to1 , ...])
                
                "births": List of ids from frame t that are 
                "deaths": List of ids.
            Ids are one-based, 0 is reserved for background.
        """
        if images is not None:
            assert len(images) == len(detections)
        else:
            images = [None] * len(detections)

        links = []
        for i in tqdm(range(len(images) - 1), desc="Linking"):
            detections0 = detections[i]
            detections1 = detections[i+1]
            self._assert_relabeled(detections0)
            self._assert_relabeled(detections1)
            
            cost_matrix = self.linking_cost_function(detections0, detections1, images[i], images[i+1])
            li = self._link_two_frames(cost_matrix)
            self._assert_links(links=li, time=i, detections0=detections0, detections1=detections1) 
            links.append(li)
            
        return links

    @abstractmethod
    def linking_cost_function(self, detections0, detections1, image0=None, image1=None):
        """Calculate features for each detection and extract pairwise costs.
        
        To be overwritten in subclass.
        
        Args:
        
            detections0: image with background 0 and detections 1, ..., m
            detections1: image with backgruond 0 and detections 1, ..., n
            image0 (optional): image corresponding to detections0
            image1 (optional): image corresponding to detections1
            
        Returns:
        
            m x n cost matrix 
        """
        pass
    
    @abstractmethod
    def _link_two_frames(self, cost_matrix):
        """Link two frames.
        
        To be overwritten in subclass.

        Args:

            cost_matrix: m x n matrix

        Returns:
        
            "links":

                Tuple of lists. Links from frame t to frame t+1 of form (from0, to0) are split up into two lists: 
                - idgs_from: [from0, from1 , ...])
                - ids_to: [to0, to1 , ...])

            "births": List of ids from frame t that are 
            "deaths": List of ids.
            
            Ids are one-based, 0 is reserved for background.
        """
        pass

    def relabel_detections(self, detections, links):
        """Relabel dense detections according to computed links, births and deaths.
        
        Args:
        
            detections: 
                 
                 List of t numpy arrays of shape (x,y) with contiguous label ids. Background = 0.
                 
            links:
                
                List of t linking dictionaries, each containing:
                    "links": Tuple of lists (ids frame t, ids frame t+1),
                    "births": List of ids,
                    "deaths": List of ids.
                Ids are one-based, 0 is reserved for background.
        """
        detections = detections.copy()
        
        assert len(detections) - 1 == len(links)
        self._assert_relabeled(detections[0])
        out = [detections[0]]
        n_tracks = out[0].max()
        lookup_tables = [{i: i for i in range(1, out[0].max() + 1)}]

        for i in tqdm(range(len(links)), desc="Recoloring detections"):
            (ids_from, ids_to) = links[i]["links"]
            births = links[i]["births"]
            deaths = links[i+1]["deaths"] if i+1 < len(links) else []
            new_frame = np.zeros_like(detections[i+1])
            self._assert_relabeled(detections[i+1])
            
            lut = {}
            for _from, _to in zip(ids_from, ids_to):
                # Copy over ID
                new_frame[detections[i+1] == _to] = lookup_tables[i][_from]
                lut[_to] = lookup_tables[i][_from]

            
            # Start new track for birth tracks
            for b in births:
                if b in deaths:
                    continue
                
                n_tracks += 1
                lut[b] = n_tracks
                new_frame[detections[i+1] == b] = n_tracks
                
            # print(lut)
            lookup_tables.append(lut)
            out.append(new_frame)
                
        return np.stack(out)

    def _assert_links(self, links, time, detections0, detections1):
        if len(links["links"][0]) != len(links["links"][1]):
            raise RuntimeError("Format of links['links'] not correct.")
            
        if sorted([*links["links"][0], *links["deaths"]]) != list(range(1, len(np.unique(detections0)))):
            raise RuntimeError(f"Some detections in frame {time} are not properly assigned as either linked or death.")
            
        if sorted([*links["links"][1], *links["births"]]) != list(range(1, len(np.unique(detections1)))):
            raise RuntimeError(f"Some detections in frame {time + 1} are not properly assigned as either linked or birth.")
            
        for b in links["births"]:
            if b in links["links"][1]:
                raise RuntimeError(f"Links frame {time+1}: Detection {b} marked as birth, but also linked.")
        
        for d in links["deaths"]:
            if d in links["links"][0]:
                raise RuntimeError(f"Links frame {time}: Detection {d} marked as death, but also linked.")
        
        
    def _assert_relabeled(self, x):
        if x.min() < 0:
            raise ValueError("Negative ID in detections.")
        if x.min() == 0:
            n = x.max() + 1
        else:
            n = x.max()
        if n != len(np.unique(x)):
            raise ValueError("Detection IDs are not contiguous.")

In [9]:
class BipartiteMatchingLinker(FrameByFrameLinker):
    """.
    
    Args:
        threshold (float): Maximum euclidian distance for linking.
        drift: tuple of (x,y) drift correction per frame.
        birth_cost_factor (float): Multiply factor with maximum entry in cost matrix.
        death_cost_factor (float): Multiply factor with maximum entry in cost matrix.
    """
    
    def __init__(
        self,
        threshold=np.finfo(float).max,
        drift=(0,0),
        birth_cost_factor=1.05,
        death_cost_factor=1.05,
        *args,
        **kwargs
    ):
        self.threshold = threshold
        self.drift = np.array(drift)
        self.birth_cost_factor = birth_cost_factor
        self.death_cost_factor = death_cost_factor
        
        super().__init__(*args, **kwargs)
        
    def linking_cost_function(self, detections0, detections1, image0=None, image1=None):
        """ Get centroids from detections and compute pairwise euclidian distances with drift correction.
                
        Args:
        
            detections0: image with background 0 and detections 1, ..., m
            detections1: image with backgruond 0 and detections 1, ..., n
            
        Returns:
        
            m x n cost matrix 
        """
        # regionprops regions are sorted by label
        regions0 = skimage.measure.regionprops(detections0)
        points0 = [np.array(r.centroid) for r in regions0]
        
        regions1 = skimage.measure.regionprops(detections1)
        points1 = [np.array(r.centroid) for r in regions1]
        
        dists = []
        for p0 in points0:
            for p1 in points1:
                dists.append(np.sqrt(((p0 + self.drift - p1)**2).sum()))

        dists = np.array(dists).reshape(len(points0), len(points1))
        
        return dists
    
    def _link_two_frames(self, cost_matrix):
        """Weighted bipartite matching with square matrix from Jaqaman et al (2008).

        Args:

            cost_matrix: m x n matrix.

        Returns:
            "links":
    
                Tuple of lists. Links from frame t to frame t+1 of form (from0, to0) are split up into two lists: 
                    - idgs_from: [from0, from1 , ...])
                    - ids_to: [to0, to1 , ...])
                
                "births": List of ids from frame t that are 
                "deaths": List of ids.
                
            Ids are one-based, 0 is reserved for background.
        """
        
        cost_matrix = cost_matrix.copy().astype(float)
        b = self.birth_cost_factor * min(self.threshold, cost_matrix.max())
        d = self.death_cost_factor * min(self.threshold, cost_matrix.max())
        no_link = max(cost_matrix.max(), max(b, d)) * 1e9
        
        cost_matrix[cost_matrix > self.threshold] = no_link
        lower_right = cost_matrix.transpose()

        deaths = np.full(shape=(cost_matrix.shape[0], cost_matrix.shape[0]), fill_value=no_link)
        np.fill_diagonal(deaths, d)
        births = np.full(shape=(cost_matrix.shape[1], cost_matrix.shape[1]), fill_value=no_link)
        np.fill_diagonal(births, b)
        
        square_cost_matrix = np.block([
            [cost_matrix, deaths],
            [births, lower_right],
        ])
        row_ind, col_ind = scipy.optimize.linear_sum_assignment(square_cost_matrix)
        
        ids_from = []
        ids_to = []
        births = []
        deaths = []
        for row, col in zip(row_ind, col_ind):
            if row < cost_matrix.shape[0] and col < cost_matrix.shape[1]:
                ids_from.append(row)
                ids_to.append(col)

            if row >= cost_matrix.shape[0] and col < cost_matrix.shape[1]:
                births.append(col)
            if row < cost_matrix.shape[0] and col >= cost_matrix.shape[1]:
                deaths.append(row)

        ids_from = np.array(ids_from)
        ids_to = np.array(ids_to)
        births = np.array(births)
        deaths = np.array(deaths)
                        
        # Account for +1 offset of the dense labels
        ids_from += 1
        ids_to += 1
        births += 1
        deaths += 1
        
        links = {"links": (ids_from, ids_to), "births": births, "deaths": deaths}
        return links

# Testing